# Cyclone Idai
Code to replicate the floodmaps of [Canty et al.](https://doi.org/10.3390/rs12010046) using this [tutorial](https://developers.google.com/earth-engine/tutorials/community/detecting-changes-in-sentinel-1-imagery-pt-4).

This method utilises information about the distribution of the pixels in the VH and VV to identify significant changes between snapshots. In all VV, VH, HV, and HH bands, water has low backscatter compared to soil and bare vegetation and so amplitude-based methods can be used to detect changes (Bonafilia 2020). This method is not so good for urban areas. 

Briefly, pixels in the $m$-look multi-look images have a gamma distribution with shape parameters $m$ and $a/m$, where $a$ is the average intensity. Change between images is tested for using likelihood ratio tests (LRTs), where a small test statistic $Q$ indicates a significant difference between image likelihoods under the two hypotheses of no change/change. The distribution of this test statistic cannot be derived analytically so is approximated using Wilk's Theorem for the $-2\log Q$ transformation, which tends to a chi-squared distribution for many measurements. Further to this, a process for identifying changes over a sequence of images (omnibus test) is derived by factorising $Q$ into intervals $R_j$ whose product is $Q$.

Cyclone Idai made landfall near Beira, Sala, Mozambique on the 15th March 2019. It caused over 1,300 deaths. The maps being replicated here range from 1st Jan 2019 to 7th June 2019.

In [1]:
# authenticate Google Earth Engine account
import ee
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AX4XfWjXT-ZRD7TJIeoFmiS-eSsJxJAG5hsoE0l7JX0q_nr0Vu-Ue1bM7F4

Successfully saved authorization token.


In [2]:
import time
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm, gamma, f, chi2
import IPython.display as disp
import folium
from functools import partial
from importlib import reload
import mortcanty_tutorial as tut
%matplotlib inline

# Add EE drawing method to folium.
def add_ee_layer(self, ee_image_object, vis_params, name, show=True):
    """Method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True,
    show = show
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [3]:
startdate = '2019-01-01'
enddate = '2019-06-07'

# From geojson.io
geoJSON = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              34.396820068359375,
              -20.015935791381146
            ],
            [
              34.778594970703125,
              -20.015935791381146
            ],
            [
              34.778594970703125,
              -19.66198700531462
            ],
            [
              34.396820068359375,
              -19.66198700531462
            ],
            [
              34.396820068359375,
              -20.015935791381146
            ]
          ]
        ]
      }
    }
  ]
}

# turn coordinates into an ee Geometry
coords = geoJSON['features'][0]['geometry']['coordinates']
aoi = ee.Geometry.Polygon(coords)

# define image collection
im_coll = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
           .filterBounds(aoi)
           .filterDate(ee.Date(startdate),ee.Date(enddate))
           .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
           .filter(ee.Filter.eq('relativeOrbitNumber_start', 6))
           .map(lambda img: img.set('date', ee.Date(img.date()).format('YYYYMMdd')))
           .sort('date'))

def clip_img(img):
    """Clip a list of images."""
    return ee.Image(img).clip(aoi)

# supply aoi to clip_img function
clip_img = partial(tut.clip_img, aoi=aoi)

im_list = im_coll.toList(im_coll.size())
im_list = ee.List(im_list.map(clip_img))

timestamplist = (im_coll.aggregate_array('date')
                 .map(lambda d: ee.String('T').cat(ee.String(d)))
                 .getInfo())

In [4]:
# Generate the set of change maps as in tutorial

# Run the algorithm with median filter and at 1% significance.
result = ee.Dictionary(tut.change_maps(im_list, median=True, alpha=0.01))
# Extract the change maps and export to assets.
cmap = ee.Image(result.get('cmap'))
smap = ee.Image(result.get('smap'))
fmap = ee.Image(result.get('fmap'))
bmap = ee.Image(result.get('bmap'))
cmaps = ee.Image.cat(cmap, smap, fmap, bmap).rename(['cmap', 'smap', 'fmap']+timestamplist[1:])

# Export to GEE then reimport and view

In [22]:
# finally, export the maps to Google Earth Engine assets
im_mean = im_coll.select('VV').mean().clip(aoi)

idai_maps = ee.Image.cat(im_mean.toDouble(),  # must have same precision
                         cmaps.select('T20190302'),
                         cmaps.select('T20190314'),
                         cmaps.select('T20190320'),
                         cmaps.select('T20190326'),
                         cmaps.select('T20190401'),
                         cmaps.select('T20190407'),
                        ).rename(['average',
                                  'T20190302',
                                  'T20190314',
                                  'T20190320',
                                  'T20190326',
                                  'T20190401',
                                  'T20190407'])

# define path to assets
assetId = 'projects/floodmapping-2022/assets/examples/idai_beira'
assexport = ee.batch.Export.image.toAsset(idai_maps,
                                          description='assetExportTask',
                                          assetId=assetId, scale=10, maxPixels=1e9)

# export to google earth engine account
assexport.start()

In [39]:
# check status of export
assexport.status()

{'state': 'COMPLETED',
 'description': 'assetExportTask',
 'creation_timestamp_ms': 1648567909016,
 'update_timestamp_ms': 1648569193517,
 'start_timestamp_ms': 1648567946409,
 'task_type': 'EXPORT_IMAGE',
 'destination_uris': ['https://code.earthengine.google.com/?asset=projects/floodmapping-2022/assets/examples/idai_beira'],
 'attempt': 1,
 'id': 'QUQF7ZHDTX6ERSS46TPHMJWS',
 'name': 'projects/earthengine-legacy/operations/QUQF7ZHDTX6ERSS46TPHMJWS'}

In [40]:
idai_maps = ee.Image('projects/floodmapping-2022/assets/examples/idai_beira')
idai_maps = idai_maps.updateMask(idai_maps.gt(0))

location = aoi.centroid().coordinates().getInfo()[::-1]
palette = ['black', 'red', 'cyan', 'yellow']

mangroves = ee.Image(ee.ImageCollection("LANDSAT/MANGROVE_FORESTS")
                               .filterBounds(aoi)
                               .first()
                               .clip(aoi))
mangrovesVis = {'min': 0, 'max': 1.0, 'palette': '#2edb7f'}

mp = folium.Map(location=location, tiles='Stamen Toner', zoom_start=11)

mp.add_ee_layer(idai_maps.select('average'), {}, 'mean of images')
mp.add_ee_layer(idai_maps.select('T20190302'), {'min': 0,'max': 3, 'palette': palette}, 'T20190302', show=False)
mp.add_ee_layer(idai_maps.select('T20190314'), {'min': 0,'max': 3, 'palette': palette}, 'T20190314', show=False)
mp.add_ee_layer(idai_maps.select('T20190320'), {'min': 0,'max': 3, 'palette': palette}, 'T20190320')
mp.add_ee_layer(idai_maps.select('T20190326'), {'min': 0,'max': 3, 'palette': palette}, 'T20190326', show=False)
mp.add_ee_layer(idai_maps.select('T20190401'), {'min': 0,'max': 3, 'palette': palette}, 'T20190401', show=False)
mp.add_ee_layer(idai_maps.select('T20190407'), {'min': 0,'max': 3, 'palette': palette}, 'T20190407', show=False)

mp.add_ee_layer(mangroves, mangrovesVis, 'mangroves')

mp.add_child(folium.LayerControl())


In [41]:
# uncomment to save the html file of the flood maps
mp.save("idai_mozambique.html")

# Export to Google Drive as GeoTIFF

In [42]:
# define path to assets
assexport_drive = ee.batch.Export.image.toDrive(idai_maps,
                                                description='assetExportTask',
                                                scale=10,
                                                maxPixels=1e9)

# export to google earth engine account
assexport_drive.start()

In [46]:
# export to google earth engine account
assexport_drive.status()

{'state': 'COMPLETED',
 'description': 'assetExportTask',
 'creation_timestamp_ms': 1648569367051,
 'update_timestamp_ms': 1648569669642,
 'start_timestamp_ms': 1648569439892,
 'task_type': 'EXPORT_IMAGE',
 'destination_uris': ['https://drive.google.com/'],
 'attempt': 1,
 'id': '2TMF5ME7DGVMM2J2BBS3TU4P',
 'name': 'projects/earthengine-legacy/operations/2TMF5ME7DGVMM2J2BBS3TU4P'}